In [1]:
import pandas as pd
import numpy as np

In [2]:
df_raw = pd.read_csv("all_games.csv")
df = df_raw.copy()

In [3]:
print(df.shape)
df.head()

(18800, 6)


,name,platform,release_date,summary,meta_score,user_review
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,"November 23, 1998","As a young boy, Link is tricked by Ganondorf, ...",99,9.1
1,Tony Hawk's Pro Skater 2,PlayStation,"September 20, 2000",As most major publishers' development efforts ...,98,7.4
2,Grand Theft Auto IV,PlayStation 3,"April 29, 2008",[Metacritic's 2008 PS3 Game of the Year; Also ...,98,7.7
3,SoulCalibur,Dreamcast,"September 8, 1999","This is a tale of souls and swords, transcendi...",98,8.4
4,Grand Theft Auto IV,Xbox 360,"April 29, 2008",[Metacritic's 2008 Xbox 360 Game of the Year; ...,98,7.9


In [4]:
df.dtypes

name            object
platform        object
release_date    object
summary         object
meta_score       int64
user_review     object
dtype: object

In [5]:
# try to convert user_review column to float
for i in df["user_review"]:
    try:
        i = float(i)
    except:
        print(i)
        break

tbd


In [6]:
# how many rows have user_review = tbd ? Drop them
print(df[df["user_review"] == "tbd"].shape)
df = df.drop(df[df["user_review"] == "tbd"].index,axis=0)
df.shape

(1365, 6)


(17435, 6)

In [7]:
# convert user_review to float
df["user_review"] = df["user_review"].astype(float)
df.dtypes

name             object
platform         object
release_date     object
summary          object
meta_score        int64
user_review     float64
dtype: object

In [8]:
# How many rows contain nans? drop them
print(df[df.isna().any(axis=1)].shape)
df = df.dropna(axis=0)
df.shape

(94, 6)


(17341, 6)

In [9]:
df.describe()

,meta_score,user_review
count,17341.000000,17341.000000
mean,71.200738,6.991269
std,12.224246,1.351931
min,20.000000,0.200000
25%,64.000000,6.300000
50%,73.000000,7.300000
75%,80.000000,7.900000
max,99.000000,9.700000


In [10]:
df.platform.unique()

array([' Nintendo 64', ' PlayStation', ' PlayStation 3', ' Dreamcast',
       ' Xbox 360', ' Wii', ' Xbox One', ' PC', ' Switch',
       ' PlayStation 2', ' PlayStation 4', ' GameCube', ' Xbox', ' Wii U',
       ' Game Boy Advance', ' 3DS', ' Xbox Series X', ' DS',
       ' PlayStation Vita', ' PlayStation 5', ' PSP', ' Stadia'],
      dtype=object)

In [11]:
# convert release_date to time
df["release_date"] = pd.to_datetime(df["release_date"])
print(df.dtypes)
df.head()

name                    object
platform                object
release_date    datetime64[ns]
summary                 object
meta_score               int64
user_review            float64
dtype: object


,name,platform,release_date,summary,meta_score,user_review
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,1998-11-23,"As a young boy, Link is tricked by Ganondorf, ...",99,9.1
1,Tony Hawk's Pro Skater 2,PlayStation,2000-09-20,As most major publishers' development efforts ...,98,7.4
2,Grand Theft Auto IV,PlayStation 3,2008-04-29,[Metacritic's 2008 PS3 Game of the Year; Also ...,98,7.7
3,SoulCalibur,Dreamcast,1999-09-08,"This is a tale of souls and swords, transcendi...",98,8.4
4,Grand Theft Auto IV,Xbox 360,2008-04-29,[Metacritic's 2008 Xbox 360 Game of the Year; ...,98,7.9


In [12]:
# What summaries are really short? They look fine
df = df.reset_index(drop=True)
print(df.shape)
for i in range(len(df["summary"])):
    if len(df["summary"][i]) < 20:
        print(df["name"][i])
        print(df["summary"][i])

(17341, 6)
The Walking Dead: Season Two Episode 3 - In Harm's Way
Consequences.
The Walking Dead: Season Two Episode 3 - In Harm's Way
Consequences.
The Walking Dead: Season Two Episode 3 - In Harm's Way
Consequences.
Age of Zombies
[PSP Minis]
The Walking Dead: Michonne - Episode 3: What We Deserve
The season finale.
The Walking Dead: Michonne - Episode 3: What We Deserve
The season finale.


In [13]:
# make summaries lower case
df['summary'] = df['summary'].str.lower()

In [14]:
# remove special characters from summary
chars = ['\[', '\]', ',', ';', '.', '!', '?', ':', '-', '\'s', '\'ll', '\'', '&', '*', ' - ']
for i in chars:
    df['summary'] = df['summary'].str.replace(i,'')

/var/folders/xx/s138h7zs36sdpnp0qvy3wk2m0000gn/T/ipykernel_37131/3206858259.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['summary'] = df['summary'].str.replace(i,'')
/var/folders/xx/s138h7zs36sdpnp0qvy3wk2m0000gn/T/ipykernel_37131/3206858259.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['summary'] = df['summary'].str.replace(i,'')


In [15]:
# what are the most common words used in all summaries?
from collections import Counter
most_freq = Counter(" ".join(df["summary"]).split()).most_common(100)
print(most_freq)

[('the', 102854), ('and', 65744), ('of', 50534), ('to', 45216), ('a', 40621), ('in', 28985), ('with', 18993), ('you', 16174), ('your', 16056), ('as', 14133), ('is', 13693), ('new', 11514), ('on', 11477), ('for', 11344), ('game', 10552), ('from', 9937), ('an', 9915), ('that', 8985), ('players', 8660), ('world', 7587), ('by', 6849), ('their', 6806), ('will', 6474), ('or', 6161), ('can', 5950), ('all', 5078), ('are', 4977), ('through', 4860), ('this', 4848), ('more', 4814), ('it', 4741), ('into', 4635), ('his', 4621), ('up', 4541), ('play', 4487), ('experience', 4023), ('be', 3869), ('take', 3620), ('has', 3564), ('have', 3341), ('battle', 3322), ('one', 3303), ('they', 3300), ('action', 3224), ('time', 3178), ('features', 3145), ('at', 3100), ('characters', 2973), ('mode', 2968), ('gameplay', 2896), ('each', 2754), ('player', 2717), ('out', 2685), ('adventure', 2651), ('unique', 2643), ('story', 2640), ('who', 2590), ('way', 2571), ('including', 2548), ('where', 2544), ('over', 2504), ('

In [16]:
# remove fluffer words
words = [' the ', ' and ', ' of ', ' to ', ' a ', ' in ', ' with ', ' your ', ' is ', ' you ', \
         ' as ', ' on ', ' for ', ' from ', ' an ', ' that ', ' by ', ' their ', ' will ', ' or ', \
         ' can ', ' all ', ' are ', ' this ', ' it ', ' into ', ' up ', ' be ', ' has ', \
         ' have ', ' its ', ' them ', ' which ', ' than ', ' but ', ' at ', ' was ', ' what ', ' when ',\
         ' take ', ' even ', ' been ', ' while ', ' who ', ' like ', ' get ', ' where ']
for i in words:
    df['summary'] = df['summary'].str.replace(i,' ')

# what are most frequent now?
most_freq = Counter(" ".join(df["summary"]).split()).most_common(100)
print(most_freq)

[('new', 11514), ('game', 10552), ('players', 8660), ('world', 7587), ('through', 4860), ('more', 4814), ('his', 4621), ('play', 4487), ('experience', 4023), ('battle', 3322), ('one', 3303), ('they', 3300), ('action', 3224), ('time', 3178), ('features', 3145), ('characters', 2973), ('mode', 2968), ('gameplay', 2896), ('each', 2754), ('player', 2717), ('out', 2685), ('adventure', 2651), ('unique', 2643), ('story', 2640), ('way', 2571), ('including', 2548), ('over', 2504), ('series', 2472), ('own', 2409), ('most', 2390), ('control', 2363), ('against', 2357), ('games', 2355), ('set', 2310), ('combat', 2310), ('online', 2243), ('multiplayer', 2191), ('first', 2174), ('must', 2159), ('system', 2141), ('he', 2131), ('use', 2131), ('weapons', 2127), ('team', 2124), ('other', 2081), ('fight', 2057), ('also', 2047), ('enemies', 2020), ('friends', 1903), ('now', 1854), ('war', 1838), ('her', 1825), ('only', 1800), ('the', 1791), ('city', 1767), ('back', 1762), ('life', 1746), ('explore', 1735), 

In [18]:
# save the cleaned df
df.to_pickle("cleaned_df.pkl")